In [21]:
### Hybird Model
import os, sys
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('../src'))
from traj_data import *
from models import *
from approximate_bayesian_new import *
#import seaborn as sns
import pandas as pd
import math

from joblib import Parallel, delayed
import timeit
import multiprocessing

In [22]:
Data_set = 'CIVIC'
#num_samples = 1000001
num_samples = 1000001
max_para_dim = 10
max_model_id = 20
max_score = 9999
ts = 0.1
result = np.empty((0,max_para_dim+5))
#for model_name in [,'IDM', 'FVM', 'GFM', 'HL', 'LLCS', 'OVM','MPC','FVM_CS','Newell','IDM_CS''FVM_SIGMOID']:
#for model_name in ['OVM']:  #,   'MPC' 
#for model_name in ['IDM']:
#for model_name in ['LL','IDM', 'FVM', 'GFM', 'HL', 'LLCS', 'OVM','MPC']: 
#for model_name in ['IDM', 'FVM', 'GFM',  'OVM']: #, 'HL'
for model_name in ['IDM', 'FVM', 'GFM',  'OVM', 'HL']: #, 'HL'
    #result_path = '../cross_ngsim/Cross_result_%d_%s_%s_1.csv' % (num_samples, model_name, Data_set)
    result_path = '../New_15_result_%d_%s_%s_new3.csv' % (num_samples, model_name, Data_set)
    temp_result =pd.read_csv(open(result_path, "rb"), delimiter=",", skiprows=1, nrows=900000)
    #temp_result = np.loadtxt(open(result_path, "rb"), delimiter=",", skiprows=1)
    #result_1w = temp_result[0:9999, :]
    # print(temp_result)
    result = np.append(result, temp_result, axis = 0)
    #result = np.append(result_1w, result_1w, axis = 0)
#np.savetxt('2vs2_result_%s.csv' % Data_set, result, delimiter = ",", fmt='%.4f')
#np.savetxt('merged_result2_%s.csv' % Data_set, result, delimiter = ",", fmt='%.4f')
np.savetxt('Single_result_%s_%s.csv' % (Data_set,  model_name), result, delimiter = ",", fmt='%.4f')

In [23]:
class particle_evaluation:
    def __init__(self, result_file, training_file, testing_file):
        self.raw_result = np.loadtxt(open(result_file, "rb"), delimiter=",", skiprows=0)
        self.training_file = training_file
        self.testing_file = testing_file
        self.load_testing_data()
    
    def load_testing_data(self):
        training_vecs = read_traj(self.training_file, ts)
        testing_vecs = read_traj(self.testing_file, ts)
        self.training_vec_leader = training_vecs[0]
        self.training_vec_follower = training_vecs[1]
        self.testing_vec_leader = testing_vecs[0]
        self.testing_vec_follower = testing_vecs[1]
        
        self.training_samples = self.training_vec_leader.num_veh
        self.testing_samples = self.testing_vec_leader.num_veh
        self.simu_len =  self.training_vec_leader.sample_points
        
    def processing_particles_result(self, weight = [0.5, 0.3, 0.2]):

        self.error_weight = weight
        score = np.zeros([self.raw_result.shape[0], 1])
        score[:,0] = self.raw_result[:,0] * weight[0]  + self.raw_result[:,1]* weight[1] + self.raw_result[:,2] * weight[2]
        result_with_score = np.concatenate((score, self.raw_result), axis = 1)
        self.result_with_score = result_with_score[result_with_score[:,0].argsort()]
        #np.savetxt('hybrid_result_with_score_%s.csv' % (Data_set), self.result_with_score, delimiter = ",", fmt='%.4f')
        np.savetxt('hybrid_result_with_score_cross_new_%s.csv' % (Data_set), self.result_with_score, delimiter = ",", fmt='%.4f')
        
        
        self.result_with_score_by_traj = {}
        for i in range(self.training_samples):
            self.result_with_score_by_traj[i] = np.empty((0,self.result_with_score.shape[1]))
        for i in range(self.result_with_score.shape[0]):
            #if self.result_with_score[i, 4] != math.inf:
            if self.result_with_score[i, 4] != math.inf:
                traj_id = int(self.result_with_score[i, 4])
            if self.result_with_score[i, 0] < max_score:
                self.result_with_score_by_traj[traj_id] = np.append(self.result_with_score_by_traj[traj_id], [self.result_with_score[i, :]], axis = 0)
        #for i in range(self.training_samples):
            #np.savetxt('hybrid_result_by_traj_%d_%s.csv' % (i, Data_set), self.result_with_score_by_traj[i], delimiter = ",", fmt='%.4f')

    def cutting_particles(self, epsilon_cut = 5):
        filtered_by_traj = {}
        model_selected_profile = np.zeros(max_model_id)
        self.particles_group_by_traj = np.empty((0,self.result_with_score.shape[1]))
        for i in range(self.training_samples):
            threshold = self.result_with_score_by_traj[i][epsilon_cut,0]
            filtered_by_traj[i] = self.result_with_score_by_traj[i][self.result_with_score_by_traj[i][:,0]<threshold]
            self.particles_group_by_traj = np.append(self.particles_group_by_traj, filtered_by_traj[i], axis = 0)
        self.n_particles = self.particles_group_by_traj.shape[0]
        self.particles_group_by_traj = self.particles_group_by_traj[self.particles_group_by_traj[:,0].argsort()]
        for i in range(self.n_particles):           
            model_selected_profile[int(self.particles_group_by_traj[i,5])] += 1
        self.particles = self.particles_group_by_traj
        np.savetxt("filtered_particles_15new_%s_%s.csv" % (Data_set, model_name), self.particles_group_by_traj, delimiter = ',', fmt = '%.4f')
        #np.savetxt("model_selection_profile2_%s.csv" % Data_set, model_selected_profile, delimiter = ',', fmt = '%d')
        #np.savetxt("model_selection2V2_profile_%s.csv" % Data_set, model_selected_profile, delimiter = ',', fmt = '%d')
        
    def cal_particle_score(self, leader, follower, particle):
        my_model = MODEL_LIST[int(particle[5])](ts)
        #print(my_model.strPara)
        simu_follower = my_model.single_simulation(leader, follower, particle[6:])
        
        #print(particle[6:])
        
        #TrajPair(leader,follower).show('a')
        #print(simu_follower.position)
        #print(simu_follower.speed)
        #print(follower.acceleration)
        
    
        error = simu_follower.distance(follower)
        
        return (self.error_weight[0]*error[0]+ self.error_weight[1]*error[1]+ self.error_weight[2]*error[2], simu_follower)
    
    def select_mean_particle(self):
        
        using_parallel = False  
        
        if using_parallel:
            
            num_cores = multiprocessing.cpu_count()
            batchsize = int(np.floor(self.training_samples / num_cores))
            n_batch = int(np.floor(self.training_samples / batchsize))
            
            print('CPU cores using: %d' % num_cores)
            print('%d Trajectories truncated' % (self.testing_samples-n_batch * batchsize))
            
            def process(n):
                grouped_score = np.zeros([self.n_particles,batchsize])
                for bn in range(batchsize):
                    loc = n*batchsize + bn                  
                    for i in range(self.n_particles):
                        grouped_score[i,bn],_ = self.cal_particle_score(self.training_vec_leader.veh[loc], self.training_vec_follower.veh[loc], self.particles[i,:])
                return grouped_score
            parallel_result_score = Parallel(n_jobs=num_cores)(delayed(process)(n) for n in range(n_batch))
            
            final_score = np.zeros([self.n_particles,n_batch*batchsize])
            for n in range(n_batch):
                for bn in range(batchsize):
                    loc = n*batchsize + bn
                    for i in range(self.n_particles): 
                        final_score[i, loc] = parallel_result_score[n][i,bn]
            
            best_particle = -1
            best_score = np.inf
            for i in range(self.n_particles): 
                particle_score = 0
                for j in range(n_batch*batchsize):
                    particle_score+=final_score[i,j]
                particle_score = particle_score / (n_batch*batchsize)
                
                if particle_score < best_score:
                    best_score = particle_score 
                    best_particle = i

            
            
        else:
            best_particle = -1
            best_score = np.inf
            for i in range(self.n_particles):
                particle = self.particles[i,:]
                particle_score = 0
                for j in range(self.training_samples):
                    temp,_ = self.cal_particle_score(self.training_vec_leader.veh[j], self.training_vec_follower.veh[j], particle)
                    particle_score += temp
                particle_score = particle_score / self.training_samples

                if particle_score < best_score:
                    best_score = particle_score 
                    best_particle = i
                    
        self.mean_particle = best_particle
        self.mean_particle_test_score = self.particle_score_array[best_particle,:].mean()
        print('best particle: %d' % best_particle)
        print('best particle score: %f' % best_score)
        return best_particle, best_score
    
    def particle_testing(self):
        self.particle_score_array = np.zeros([self.n_particles, self.testing_samples])
        self.sim_traj = {}
        num_cores = multiprocessing.cpu_count()

        batchsize = int(np.floor(self.testing_samples / num_cores))
        n_batch = int(np.floor(self.testing_samples / batchsize))
        
        using_parallel = False    ####switch
        
        if using_parallel:
            print('CPU cores using: %d' % num_cores)
            print('%d Trajectories truncated' % (self.testing_samples-n_batch * batchsize))
            def process(n):
                grouped_score = np.zeros([self.n_particles,batchsize])
                grouped_sim_traj = {}
                for bn in range(batchsize):
                    loc = n*batchsize + bn                  
                    for i in range(self.n_particles):
                        grouped_score[i,bn],grouped_sim_traj[i,bn] = self.cal_particle_score(self.testing_vec_leader.veh[loc], self.testing_vec_follower.veh[loc], self.particles[i,:])
                return grouped_score,grouped_sim_traj
    
            parallel_result_zipped = Parallel(n_jobs=num_cores)(delayed(process)(n) for n in range(n_batch))
            parallel_result_score, parallel_result_sim_traj = zip(*parallel_result_zipped)
            
            for n in range(n_batch):
                for bn in range(batchsize):
                    loc = n*batchsize + bn
                    for i in range(self.n_particles): 
                        self.particle_score_array[i, loc] = parallel_result_score[n][i,bn]
                        self.sim_traj[i, loc] = parallel_result_sim_traj[n][i,bn]
        #self.testing_vec_leader
        else:
            for i in range(self.n_particles):
                particle = self.particles[i,:]            
                for j in range(self.testing_samples):
                    self.particle_score_array[i, j], self.sim_traj[i, j] = self.cal_particle_score(self.testing_vec_leader.veh[j], self.testing_vec_follower.veh[j], particle)
                    
                    ####Debug
                    #if i == 0 and j==0:
                        #TrajPair(self.testing_vec_leader.veh[j],self.testing_vec_follower.veh[j]).show('s')
                        #print(particle)
                        #print(self.particle_score_array[i, j])
                    
        #np.savetxt('Score_array_%s_%s.csv' % (Data_set, model_name), self.particle_score_array, delimiter = ',', fmt ='%.4f')
                
    def cal_wasserstein_dist(self):
        C = self.particle_score_array
        #print(C)
        #np.savetxt('Score_array.csv', C, delimiter = ',', fmt ='%.4f')
        X = cp.Variable(C.shape)
        E1 = np.ones(C.shape[1])
        E0 = np.ones(C.shape[0])
        prob = cp.Problem(cp.Minimize(cp.trace(C @ X.T)), 
                         [X @ E1 == 1/C.shape[0],
                         X.T @ E0 == 1/C.shape[1],
                         X >= 0,
                         X <= 1])
        prob.solve()
        #prob.solve(solver=cp.SCS, verbose = True)
        #prob.solve(solver = cp.SCIPY, verbose = True)
        self.X = X.value
        self.wasserstein_dist = prob.value 
        #np.savetxt("Probability array_IDM.csv",X.value, delimiter = ',', fmt ='%.4f')

#### Stochastic Metrics        
    def cal_minimum_dist(self):
        C = self.particle_score_array
        X = cp.Variable(C.shape)
        E1 = np.ones(C.shape[1])
        E0 = np.ones(C.shape[0])
        beta = 0
        prob = cp.Problem(cp.Minimize(cp.trace(C @ X.T)), 
                         [#X @ E1 == 1/C.shape[0],    ####shape 0: row
                         X.T @ E0 == 1/C.shape[1],
                         X >= 0,
                         X <= 1])
        prob.solve()
        self.X = X.value
        self.minimum_dist = prob.value 
       

    def cal_five_percent_dist(self):
        C = self.particle_score_array
        X = cp.Variable(C.shape)
        E1 = np.ones(C.shape[1])
        E0 = np.ones(C.shape[0])
        beta = 0.15
        prob = cp.Problem(cp.Minimize(cp.trace(C @ X.T)), 
                         [X @ E1 >= beta*(1/C.shape[0]),
                         X.T @ E0 == (1/C.shape[1]),
                         X >= 0,
                         X <= 1])
        prob.solve()
        self.X = X.value
        self.five_percent_dist = prob.value 
        
           
        
#### New metric        
    #def expectation(self):

        #C = self.particle_score_array
        #C_bar = C.mean(axis = 1)
        #E1 = np.ones(C_bar.shape[0])
        #Solution = np.zeros(len(C_bar))
        #Number = np.zeros(len(C_bar))
        
        #Y = cp.Variable(C_bar.shape, integer = True)
        #for N in range (1, len(C_bar)):
        ##N = 2
            #prob = cp.Problem(cp.Minimize(cp. sum((C_bar @ Y)/N)),
                               # [Y @ E1 == N,
                                 #Y >= 0,
                                # Y <= 1])
            #prob.solve()
            #self.Y = Y.value
            #self.expectation = prob.value
            #Solution[N - 1] =prob.value
            ##print(Y.value)
            ##solu = np.concatenate(Solution, Number)
            
        ##np.savetxt('Exp.csv', Solution, delimiter = ',', fmt ='%.4f')
        ##return (min(Solution))
        
        
        
    def get_particle_quantile(self, veh, n):
        #  return the index of the nth minimum score for a particular vehicle      
        score = self.particle_score_array[:, veh]
        val = np.partition(score,n)[n]
        return np.where(score == val)[0][0]
    
    
    
    
    
    def get_veh_quantile(self, particle, n):
        #  return the index of the nth minimum score for a particular vehicle      
        score = self.particle_score_array[particle, :]
        val = np.partition(score,n)[n]
        return np.where(score == val)[0][0]
    
    # def show_best_particle(self):
    #     for j in range(self.testing_samples):
    #         plt.plot(self.sim_traj[self.best_particle, j].position - self.testing_vec_follower.veh[j].position, 'b.', markersize=.03)
    #     plt.show()
    
    def error_plot(self, particle, vehicle, d_type, marker, markersize):
        if d_type == 'p':
            plt.plot(self.sim_traj[particle, vehicle].position - self.testing_vec_follower.veh[vehicle].position, marker, markersize=markersize)
            return(self.sim_traj[particle, vehicle].position - self.testing_vec_follower.veh[vehicle].position)
        if d_type == 's':
            plt.plot(self.sim_traj[particle, vehicle].speed - self.testing_vec_follower.veh[vehicle].speed, marker, markersize=markersize)
            return(self.sim_traj[particle, vehicle].speed - self.testing_vec_follower.veh[vehicle].speed)
        if d_type == 'a':
            plt.plot(self.sim_traj[particle, vehicle].acceleration - self.testing_vec_follower.veh[vehicle].acceleration, marker, markersize=markersize)
            return(self.sim_traj[particle, vehicle].acceleration - self.testing_vec_follower.veh[vehicle].acceleration)
        
        
    def show_all_particles(self, veh_id, d_type, marker, markersize):
        for i in range(self.n_particles):
            self.error_plot(i, veh_id, d_type, marker, markersize)
    
    def show_all_vehs_of_particle(self, particle_select, d_type, marker, markersize):
        for i in range(self.testing_samples):
            self.error_plot(particle_select, i, d_type, marker, markersize)
    
    def show_nth_best_veh_of_particle(self, particle_id, n, d_type, marker, markersize):
        veh = self.get_veh_quantile(particle_id, n)
        self.error_plot(particle_id, veh, d_type, marker, markersize)
        return veh
                   
    def show_nth_best_particle_of_veh(self, veh_id, n, d_type, marker):
        particle_select = self.get_particle_quantile(veh_id, n)
        self.error_plot(particle_select, veh_id, d_type, marker, 1)
        return particle_select
    
    def show_best_n_particle_of_veh(self, veh_id, n, d_type, marker):
        plot_result = {}
        for i in range(n):
            particle_select = self.get_particle_quantile(veh_id, i)
            plot_result[i] = self.error_plot(particle_select, veh_id, d_type, marker, 1)
        return plot_result
        
    def show_matched_particles_of_veh(self, veh_id, d_type, marker):
        n_list = np.where(self.X[:,veh_id]>1e-5)[0]
        for i in n_list:
            particle_select = i
            self.error_plot(particle_select, veh_id, d_type, marker, 1)
    def show_matched_particles_weight_of_veh(self, veh_id, d_type, marker):
        for i in range(self.n_particles):
            particle_select = i
            self.error_plot(particle_select, veh_id, d_type, marker, markersize = self.X[i,veh_id]/100)
       
    def show_mean_particle_of_veh(self, veh_id, d_type, marker):
        self.error_plot(self.mean_particle, veh_id, d_type, marker, 1)

In [24]:
weight = [0.5, 0.3, 0.2]
#weight = [1, 0, 0]
#my_eval = particle_evaluation('merged_result2_%s.csv' % Data_set,
my_eval = particle_evaluation('Single_result_%s_%s.csv' % (Data_set,  model_name),
                              '../data/%s/Trajectory/' % Data_set,
                              '../data/%s/Testing/' % Data_set)
                              #'../data/%s/Trajectory/' % Data_set)
my_eval.processing_particles_result(weight)
my_eval.cutting_particles(3) ##default as 5

In [ ]:
my_eval.load_testing_data()
tic = timeit.default_timer()
my_eval.particle_testing()
toc = timeit.default_timer()
print('Time consumption (Particle Testing) %d sec ' % (toc - tic))
tic = timeit.default_timer()
my_eval.select_mean_particle()
toc = timeit.default_timer()
print('Time consumption (Mean Particle Selection) %d sec ' % (toc - tic))
my_eval.cal_wasserstein_dist()
my_eval.cal_minimum_dist()
my_eval.cal_five_percent_dist() 
#my_eval.std()
#print(my_eval.five_percent_dist)
print(my_eval.mean_particle_test_score, my_eval.minimum_dist, my_eval.five_percent_dist, my_eval.wasserstein_dist)

In [ ]:
veh = 2
LABEL = {'p': 'Position Error (m)', 's': 'Speed Error (m/s)', 'a':''}
def show_fitted_result(my_eval, veh, d_type, yr):
    
    #my_eval.show_mean_particle_of_veh(veh,d_type,'r')
    # my_eval.show_matched_particles_of_veh(veh,d_type,'b.')
    #my_eval.show_nth_best_particle_of_veh(veh,0,d_type,'b')
    #my_eval.show_nth_best_particle_of_veh(veh,5,d_type,'g')
    my_eval.show_best_n_particle_of_veh(veh,5,d_type,'r')
    
    # my_eval.show_all_particles_testing_veh(veh)
    # plt.ylim(yr)
    
    # plt.show()
    plt.plot([0,0],[0,0],'y')
    my_eval.show_all_particles(veh,d_type, 'y.', .04)
    #plt.legend(['Top 5% Best Fitted Particles of Hybrid model','All Particles'])  #,  
    plt.ylim(yr)
    plt.xlabel('Time Interval (0.1 sec)')
    plt.ylabel(LABEL[d_type])
    #plt.savefig(d_type + '_veh_%d_%s.jpg' % (veh, Data_set), dpi = 1200) #model_name
    #plt.savefig(d_type + '_veh_%d_hybrid_Model_Data_%s_%s.jpg' % (veh, Data_set, model_name), dpi = 1200) #model_name,
    plt.show()

show_fitted_result(my_eval, veh, 'p', [-15, 15])
#show_fitted_result(my_eval, veh, 's', [-8, 8])
show_fitted_result(my_eval, veh, 'p', [-10, 10])
#show_fitted_result(my_eval, veh, 's', [-10, 10])
#print(my_eval.mean_particle_test_score, my_eval.expectation)
print(my_eval.wasserstein_dist, my_eval.mean_particle_test_score)
#print(my_eval.wasserstein_dist, my_eval.mean_particle_test_score, my_eval.expectation)

In [ ]:
import seaborn as sns
n_dim = 6
particles = np.loadtxt('C:/Users/jjiang284/Box/Hybrid model/hybrid/Filtered/tesla/Tesla_analysis_hl.csv', delimiter=",", skiprows=0)
for i in range(n_dim):
    ax=sns.distplot(particles[:,i], bins=10, color='red',
    hist_kws={"edgecolor": 'black'})
    #xlimit = ([0,2],
              #[-0.2,1],
              #[-0.2,1],
              #[-0.2,0.8],
              #[-4,-1],
              #[2,9])
    xlabel = ['tau: Desired time Gap (sec)', 'TT: Actuation lag (sec)', 'Ks: Spacing deviation feedback gain', 'Kv: Speed difference feedback gain','Ka: Acceleration feedback gain','l: standstill distance (m)']  ##HL
    #xlabel = ['a: maximum accleration (m/s2)', 'b: desired deceleration (m/s2)', 'Vmax: desired speed (m/s)','T: desired time gap (sec)', 's0: minimum gap (m)', 'delta: free accleration exponent'] #IDM
    plt.title('Calibrated',fontsize=18)
    plt.xlabel(xlabel[i],fontsize=18)
    plt.ylabel('Density',fontsize=18)
    plt.savefig('HL_syn_gen_1m{}.jpg'.format(i), dpi = 1200)
    plt.show()
    


In [ ]:
pip install cvxopt

In [ ]:
test_a = np.ones([3,4])
test_a[1][1]=0
print(test_a)

In [ ]:
score = test_a.sum(axis=1)
print(min(score))

In [ ]:
print(np.amin(score))
print(np.where(score == np.amin(score)))

In [ ]:
x = np.linspace(0, 2, 100)  # Sample data.

plt.figure(figsize=(5, 2.7))
plt.plot(x, x, 'b',label='Top 5% Best Fitted Particles of GFM')  # Plot some data on the (implicit) axes.
plt.plot(x, x**2, 'g', label='Top 5% Best Fitted Particles of HL')  # etc.
plt.plot(x, x**3, 'm', label='Top 5% Best Fitted Particles of IDM')
plt.plot(x, x**4, 'y', label='All Particles')

plt.xlabel('x label')
plt.ylabel('y label')
plt.title("Simple Plot")
plt.legend();
plt.savefig('legends.jpg', dpi = 800)

In [ ]:
x = np.linspace(0, 2, 100)  # Sample data.

plt.figure(figsize=(10, 2.7))
plt.plot(x, x, 'r',label='Top 5% Best Fitted Particles of Hybrid model')  # Plot some data on the (implicit) axes.
plt.plot(x, x**4, 'y', label='All Particles')

plt.xlabel('x label')
plt.ylabel('y label')
plt.title("Simple Plot")
plt.legend();
plt.savefig('legends2.jpg', dpi = 800)